### Tutorial Brian2 para Izhikevich - 29.7.25

Aspectos que Brian2 maneja elegantemente:

- Ecuaciones diferenciales del modelo neuronal
- Condiciones de reset post-spike
- Conectividad espacialmente estructurada
- Monitoreo de spikes y variables de estado
- Paralelización automática

Ventajas clave:

- Soporte nativo para modelos de Izhikevich
- Manejo automático de delays axonales
- STDP implementado como funcionalidad built-in
- Sistema de unidades físicas integrado (ms, mV, etc.)
- Optimización automática del código generado

El framework te permitirá enfocarte en la lógica del modelo sin preocuparte por optimizaciones de bajo nivel. La sintaxis declarativa hace el código muy legible y modificable.

-----

#### Conceptos fundamentales de Brian2

Objetos principales:

- NeuronGroup: Poblaciones de neuronas con ecuaciones diferenciales
- Synapses: Conexiones sinápticas con plasticidad
- SpikeMonitor/StateMonitor: Registro de actividad
- run(): Ejecuta la simulación

Flujo típico:

- Definir ecuaciones neuronales
- Crear grupos de neuronas
- Establecer conectividad
- Configurar monitoreo
- Ejecutar y analizar

-----

Estrategia de implementación progresiva:

1. Red pequeña inicial (1000-5000 neuronas) para debugging rápido
2. Modelo neuronal básico primero, sin plasticidad
3. Conectividad simple con probabilidades fijas
4. Añadir delays de conducción axonal
5. Monitoreo y visualización en cada paso
6. .... Implementar STDP gradualmente
   
-----

In [ ]:
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt

start_scope()

# Parámetros exactos del paper
N_exc = 800  # 80% de 1000 neuronas como en el código MATLAB
N_inh = 200  # 20% de 1000 neuronas
N_total = N_exc + N_inh

# Ecuaciones exactas del paper
equations = '''
dv/dt = (0.04*v**2 + 5*v + 140 - u + I_syn + I_thalamic)/ms : 1
du/dt = a*(b*v - u)/ms : 1
I_syn : 1
I_thalamic : 1
a : 1
b : 1  
c : 1
d : 1
'''

# Crear grupos separados
exc_neurons = NeuronGroup(N_exc, equations, 
                          threshold='v >= 30',
                          reset='v = c; u += d',
                          method='euler')

inh_neurons = NeuronGroup(N_inh, equations,
                          threshold='v >= 30', 
                          reset='v = c; u += d',
                          method='euler')

# Heterogeneidad EXACTA del paper
r_exc = np.random.rand(N_exc)
r_inh = np.random.rand(N_inh)

# Excitatorias: (a,b) = (0.02, 0.2), (c,d) = (-65, 8) + (15, -6)r²
exc_neurons.a = 0.02
exc_neurons.b = 0.2
exc_neurons.c = -65 + 15 * r_exc**2
exc_neurons.d = 8 - 6 * r_exc**2

# Inhibitorias: (a,b) = (0.02, 0.25) + (0.08, -0.05)r, (c,d) = (-65, 2)
inh_neurons.a = 0.02 + 0.08 * r_inh
inh_neurons.b = 0.25 - 0.05 * r_inh
inh_neurons.c = -65
inh_neurons.d = 2

# Condiciones iniciales
exc_neurons.v = -65
exc_neurons.u = exc_neurons.b * exc_neurons.v
inh_neurons.v = -65
inh_neurons.u = inh_neurons.b * inh_neurons.v

# CONECTIVIDAD exacta del paper
# Matriz S: [0.5*rand(Ne+Ni,Ne), -rand(Ne+Ni,Ni)]

# 1. Excitatorias -> Excitatorias
syn_ee = Synapses(exc_neurons, exc_neurons, 'w : 1', on_pre='I_syn_post += w')
syn_ee.connect(p=0.09)
syn_ee.w = '0.5 * rand()'
syn_ee.delay = 'rand() * 12*ms'

# 2. Excitatorias -> Inhibitorias  
syn_ei = Synapses(exc_neurons, inh_neurons, 'w : 1', on_pre='I_syn_post += w')
syn_ei.connect(p=0.09)
syn_ei.w = '0.5 * rand()'
syn_ei.delay = 'rand() * 12*ms'

# 3. Inhibitorias -> Excitatorias
syn_ie = Synapses(inh_neurons, exc_neurons, 'w : 1', on_pre='I_syn_post -= w')
syn_ie.connect(p=0.09)
syn_ie.w = 'rand()'
syn_ie.delay = 'rand() * 2*ms'

# 4. Inhibitorias -> Inhibitorias
syn_ii = Synapses(inh_neurons, inh_neurons, 'w : 1', on_pre='I_syn_post -= w')
syn_ii.connect(p=0.09)
syn_ii.w = 'rand()'
syn_ii.delay = 'rand() * 2*ms'

# Input talámico exacto del paper: I = [5*randn(Ne,1); 2*randn(Ni,1)]
# Cada ms, cada neurona recibe input Gaussiano
thalamic_exc = PoissonInput(exc_neurons, 'I_thalamic', N=1, rate=1000*Hz, weight='5*randn()')
thalamic_inh = PoissonInput(inh_neurons, 'I_thalamic', N=1, rate=1000*Hz, weight='2*randn()')

# Monitoreo
spike_mon_exc = SpikeMonitor(exc_neurons)
spike_mon_inh = SpikeMonitor(inh_neurons)
state_mon_exc = StateMonitor(exc_neurons, ['v'], record=range(0, min(100, N_exc), 10))

print("Simulando modelo exacto del paper...")
run(1000*ms)

# Análisis como en el paper
figure(figsize=(15, 10))

# 1. Raster plot estilo paper
subplot(2, 3, 1)
plot(spike_mon_exc.t/ms, spike_mon_exc.i, '.k', markersize=0.5)
plot(spike_mon_inh.t/ms, spike_mon_inh.i + N_exc, '.k', markersize=0.5)
axhline(y=N_exc, color='r', linestyle='-', linewidth=1)
xlabel('Tiempo (ms)')
ylabel('Neurona')
title('Raster Plot - Estilo Paper')
ylim(0, N_total)

# 2. Voltaje de neurona típica
subplot(2, 3, 2)
neuron_idx = 0
plot(state_mon_exc.t/ms, state_mon_exc.v[neuron_idx], 'k-', linewidth=1)
xlabel('Tiempo (ms)')
ylabel('Voltaje (mV)')
title('Neurona Excitatoria Típica')
ylim(-80, 40)

# 3. Distribución de parámetros c,d (como Fig. 2 del paper)
subplot(2, 3, 3)
scatter(exc_neurons.c, exc_neurons.d, s=1, alpha=0.6, c='blue', label='Excitatorias')
scatter(inh_neurons.c, inh_neurons.d, s=1, alpha=0.6, c='red', label='Inhibitorias')
xlabel('Parámetro c (mV)')
ylabel('Parámetro d')
title('Distribución de Parámetros (Fig. 2)')
legend()

# 4. Histograma de frecuencias
subplot(2, 3, 4)
freq_exc = []
freq_inh = []

for i in range(N_exc):
    spike_count = sum(spike_mon_exc.i == i)
    freq_exc.append(spike_count)

for i in range(N_inh):
    spike_count = sum(spike_mon_inh.i == i) 
    freq_inh.append(spike_count)

hist(freq_exc, bins=30, alpha=0.7, density=True, label='Excitatorias')
hist(freq_inh, bins=30, alpha=0.7, density=True, label='Inhibitorias')
xlabel('Spikes en 1000ms')
ylabel('Densidad')
title('Distribución de Actividad')
legend()

# 5. Actividad poblacional (como Fig. 3 del paper)
subplot(2, 3, 5)
bin_size = 10*ms
time_bins = np.arange(0, 1000, bin_size/ms)
activity = []

for t in time_bins:
    total_spikes = (sum((spike_mon_exc.t >= t*ms) & (spike_mon_exc.t < (t + bin_size/ms)*ms)) +
                   sum((spike_mon_inh.t >= t*ms) & (spike_mon_inh.t < (t + bin_size/ms)*ms)))
    activity.append(total_spikes)

plot(time_bins, activity, 'k-', linewidth=1)
xlabel('Tiempo (ms)')
ylabel(f'Spikes/{bin_size/ms:.0f}ms')
title('Actividad de Red')

# 6. Análisis espectral simple
subplot(2, 3, 6)
from scipy import signal
if len(activity) > 100:
    freqs, psd = signal.periodogram(activity, fs=1000/(bin_size/ms))
    plot(freqs[1:50], psd[1:50])
    xlabel('Frecuencia (Hz)')
    ylabel('PSD')
    title('Espectro de Actividad')
    xlim(0, 100)

tight_layout()
show()

# Estadísticas exactas como en el paper
print(f"\n{'='*60}")
print("COMPARACIÓN CON PAPER ORIGINAL")
print(f"{'='*60}")
print(f"Arquitectura:")
print(f"  Neuronas: {N_exc} exc + {N_inh} inh (ratio 4:1 ✓)")
print(f"  Conexiones: {len(syn_ee.i) + len(syn_ii.i)+ len(syn_ie.i)+ len(syn_ei.i)} totales")
print(f"  Prob. conexión: {len(syn_ee.i)/(N_total*N_exc):.3f} (paper: ~0.09)")

print(f"\nActividad:")
total_spikes_exc = len(spike_mon_exc.t)
total_spikes_inh = len(spike_mon_inh.t)
mean_freq_exc = total_spikes_exc / N_exc  # spikes por neurona en 1s
mean_freq_inh = total_spikes_inh / N_inh

print(f"  Freq. excitatorias: {mean_freq_exc:.1f} Hz (paper: ~8Hz)")
print(f"  Freq. inhibitorias: {mean_freq_inh:.1f} Hz")
print(f"  Spikes totales: {total_spikes_exc + total_spikes_inh}")

print(f"\nParámetros (verificación aleatoria):")
idx = np.random.randint(0, N_exc)
print(f"  Neurona exc #{idx}: c={exc_neurons.c[idx]:.1f}, d={exc_neurons.d[idx]:.1f}")
idx = np.random.randint(0, N_inh)  
print(f"  Neurona inh #{idx}: a={inh_neurons.a[idx]:.3f}, b={inh_neurons.b[idx]:.3f}")

print(f"\nComportamiento emergente:")
asynchrony = np.std(activity) / np.mean(activity)
print(f"  Índice asincronía: {asynchrony:.2f} (>1 = asíncrono ✓)")
active_exc = sum(np.array(freq_exc) > 0)
print(f"  Neuronas activas: {active_exc}/{N_exc} ({100*active_exc/N_exc:.1f}%)")

------

#### Metodología del Paper para 1 Población

##### Estructura Espacial
- Geometría: Neuronas distribuidas aleatoriamente en superficie esférica (radio 8mm)
- Densidad: 125 neuronas/mm² típica cortical
- Ratio: 80% excitatorias, 20% inhibitorias
##### Conectividad Dependiente de Distancia
- Excitatorias: 75 conexiones locales (radio 1.5mm) + 25 distantes (0.5mm)
- Inhibitorias: 25 conexiones locales únicamente (radio 0.5mm)
- Probabilidad local: 0.09 entre excitatorias cercanas
##### Delays de Conducción Críticos
- Axones mielinizados: 1 m/s → delays hasta 12ms para conexiones largas
- Colaterales locales: 0.15 m/s → delays hasta 10ms
- Cálculo: delay = distancia_euclidiana / velocidad
##### Parámetros Neuronales Específicos
- Heterogeneidad continua: r² para sesgo hacia RS
- Distribución exacta: Como código MATLAB del paper
##### Input Talámico
- Poisson: q=1Hz por neurona
- Amplitud: Gaussiano (σ=5 exc, σ=2 inh)
##### Objetivo Clave
- La interacción entre delays heterogéneos y timing de spikes genera auto-organización en grupos sin STDP explícito inicialmente.

#### Paso 1: Geometría Espacial

Implementar:
- Distribución en Esfera

- Radio: 8mm
- Coordenadas (x,y,z) aleatorias en superficie
- 1000 neuronas: 800 exc, 200 inh

Cálculo de Distancias

- Distancia euclidiana/geodésica entre pares de neuronas
- Matriz de distancias 1000×1000

Visualización

- Proyección 2D de posiciones
- Verificar distribución uniforme

Conceptos Brian2 nuevos:

- Arrays numpy para coordenadas
- Funciones de distancia
- Indexing espacial

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Parámetros del paper
N_exc = 8000
N_inh = 2000
N_total = N_exc + N_inh
sphere_radius = 8.0  # mm

def generate_sphere_points(n_points, radius):
    """Genera puntos aleatorios uniformemente distribuidos en superficie esférica"""
    # Método de Marsaglia (1972) para distribución uniforme en esfera
    points = np.zeros((n_points, 3))
    
    for i in range(n_points):
        # Generar punto uniforme en esfera unitaria
        while True:
            x1, x2 = np.random.uniform(-1, 1, 2)
            if x1**2 + x2**2 < 1:
                break
        
        # Convertir a coordenadas cartesianas en esfera
        points[i, 0] = 2 * x1 * np.sqrt(1 - x1**2 - x2**2)  # x
        points[i, 1] = 2 * x2 * np.sqrt(1 - x1**2 - x2**2)  # y  
        points[i, 2] = 1 - 2 * (x1**2 + x2**2)              # z
    
    return points * radius

def geodesic_distance(pos1, pos2, radius):
    # Normalizar a vectores unitarios
    v1 = pos1 / np.linalg.norm(pos1)
    v2 = pos2 / np.linalg.norm(pos2)
    # Ángulo central
    cos_angle = np.clip(np.dot(v1, v2), -1, 1)
    angle = np.arccos(cos_angle)
    # Distancia sobre superficie
    return radius * angle

def calculate_distance_matrix(positions, r):
    """Calcula matriz de distancias euclidiana"""
    n = len(positions)
    distances = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i+1, n):
            # dist = np.linalg.norm(positions[i] - positions[j])
            dist = geodesic_distance(positions[i], positions[j], r)
            distances[i, j] = distances[j, i] = dist
    
    return distances

# Generar posiciones en esfera
print("Generando posiciones neuronales en esfera...")
neuron_positions = generate_sphere_points(N_total, sphere_radius)

# Separar excitatorias e inhibitorias
exc_positions = neuron_positions[:N_exc]
inh_positions = neuron_positions[N_exc:]

# Calcular matriz de distancias
print("Calculando matriz de distancias...")
distance_matrix = calculate_distance_matrix(neuron_positions, sphere_radius)

# Análisis de distancias
distances_flat = distance_matrix[np.triu_indices_from(distance_matrix, k=1)]
mean_distance = np.mean(distances_flat)
max_distance = np.max(distances_flat)

print(f"Estadísticas espaciales:")
print(f"  Radio esfera: {sphere_radius} mm")
print(f"  Distancia promedio: {mean_distance:.2f} mm")
print(f"  Distancia máxima: {max_distance:.2f} mm")
print(f"  Diámetro teórico: {2*sphere_radius:.1f} mm")

# Visualización
fig = plt.figure(figsize=(15, 5))

# 1. Vista 3D de la esfera
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(exc_positions[:, 0], exc_positions[:, 1], exc_positions[:, 2], 
           c='blue', s=8, alpha=0.6, label=f'Excitatorias ({N_exc})')
ax1.scatter(inh_positions[:, 0], inh_positions[:, 1], inh_positions[:, 2], 
           c='red', s=12, alpha=0.8, label=f'Inhibitorias ({N_inh})')

# Dibujar esfera wireframe
u = np.linspace(0, 2 * np.pi, 20)
v = np.linspace(0, np.pi, 20)
x_sphere = sphere_radius * np.outer(np.cos(u), np.sin(v))
y_sphere = sphere_radius * np.outer(np.sin(u), np.sin(v))
z_sphere = sphere_radius * np.outer(np.ones(np.size(u)), np.cos(v))
ax1.plot_wireframe(x_sphere, y_sphere, z_sphere, alpha=0.1, color='gray')

ax1.set_xlabel('X (mm)')
ax1.set_ylabel('Y (mm)')
ax1.set_zlabel('Z (mm)')
ax1.set_title('Distribución 3D en Esfera')
ax1.legend()

# 2. Proyección 2D (vista superior)
ax2 = fig.add_subplot(132)
ax2.scatter(exc_positions[:, 0], exc_positions[:, 1], 
           c='blue', s=8, alpha=0.6, label='Excitatorias')
ax2.scatter(inh_positions[:, 0], inh_positions[:, 1], 
           c='red', s=12, alpha=0.8, label='Inhibitorias')

# Círculo proyectado
circle = plt.Circle((0, 0), sphere_radius, fill=False, color='gray', alpha=0.3)
ax2.add_patch(circle)

ax2.set_xlabel('X (mm)')
ax2.set_ylabel('Y (mm)')
ax2.set_title('Proyección 2D (plano XY)')
ax2.set_aspect('equal')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Histograma de distancias
ax3 = fig.add_subplot(133)
ax3.hist(distances_flat, bins=50, alpha=0.7, density=True, color='green')
ax3.axvline(mean_distance, color='red', linestyle='--', 
           label=f'Media: {mean_distance:.2f} mm')
ax3.axvline(2*sphere_radius, color='black', linestyle='--', 
           label=f'Diámetro: {2*sphere_radius:.1f} mm')
ax3.set_xlabel('Distancia (mm)')
ax3.set_ylabel('Densidad')
ax3.set_title('Distribución de Distancias')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Verificación de uniformidad
print(f"\nVerificación de uniformidad:")
print(f"  Densidad teórica: 125 neuronas/mm² (área ~804 mm²)")
surface_area = 4 * np.pi * sphere_radius**2
actual_density = N_total / surface_area
print(f"  Densidad actual: {actual_density:.1f} neuronas/mm²")

# Análisis de vecindarios (preparación para conectividad)
local_radius = 1.5  # mm para excitatorias
distant_radius = 0.5  # mm para conexiones distantes

print(f"\nAnálisis de vecindarios:")
# Contar vecinos locales para algunas neuronas
sample_neurons = np.random.choice(N_exc, 5, replace=False)
for neuron_id in sample_neurons:
    distances_to_neuron = distance_matrix[neuron_id, :N_exc]  # Solo excitatorias
    local_neighbors = np.sum(distances_to_neuron <= local_radius) - 1  # -1 para excluir la neurona misma
    print(f"  Neurona {neuron_id}: {local_neighbors} vecinos en {local_radius}mm")

# Exportar datos para siguiente paso
np.save('neuron_positions.npy', neuron_positions)
np.save('distance_matrix.npy', distance_matrix)
print(f"\nDatos guardados: neuron_positions.npy, distance_matrix.npy")

#### Próximo paso: Conectividad dependiente de distancia usando esta matriz geodésica.

##### Reglas Completas de Conectividad del Paper

##### Arquitectura General

- 100,000 neuronas: 80,000 excitatorias + 20,000 inhibitorias (ratio 4:1)
- Distribución aleatoria en superficie esférica (radio 8mm)
- Densidad: ~125 neuronas/mm²

##### Neuronas Excitatorias

Conexiones locales:

- 75 targets dentro de radio 1.5mm
- Selección aleatoria entre candidatos disponibles
- Probabilidad de conexión entre excitatorias cercanas: 0.09

Conexiones distantes:

- 1 axón mielinizado recto de 12mm a ubicación aleatoria
- 25 targets en colaterales dentro de 0.5mm de esa ubicación distante
- Si <25 candidatos disponibles, conectar todos los posibles

##### Neuronas Inhibitorias

- 25 conexiones dentro de radio 0.5mm
- Targets: cualquier neurona (excitatoria o inhibitoria)
- Selección aleatoria entre candidatos locales

##### Restricciones

- Sin auto-conexiones
- Cada conexión es única (no múltiples entre mismo par)
- Inhibitorias solo conexiones locales (sin axones distantes)

        Totales Esperados por Neurona

        Excitatorias: ~100 salientes (75 local + 25 distante + conexiones E→I)
        Inhibitorias: ~25 salientes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Cargar datos espaciales
neuron_positions = np.load('neuron_positions.npy')
distance_matrix = np.load('distance_matrix.npy')

N_exc = 800
N_inh = 200
N_total = N_exc + N_inh

def generate_distant_location(neuron_pos, target_distance):
    """Genera ubicación aleatoria a distancia específica sobre superficie esférica"""
    # Normalizar posición neuronal a vector unitario
    neuron_unit = neuron_pos / np.linalg.norm(neuron_pos)
    
    # Ángulo correspondiente a la distancia sobre esfera radio 8mm
    angle = target_distance / 8.0  # distancia geodésica / radio
    
    # Generar dirección aleatoria perpendicular
    random_vec = np.random.randn(3)
    random_vec = random_vec - np.dot(random_vec, neuron_unit) * neuron_unit
    random_vec = random_vec / np.linalg.norm(random_vec)
    
    # Rotar neurona hacia ubicación distante
    distant_unit = np.cos(angle) * neuron_unit + np.sin(angle) * random_vec
    distant_location = distant_unit * 8.0  # proyectar a superficie
    
    return distant_location

def create_spatial_connectivity():
    """Crea conectividad según reglas del paper"""
    
    # Lista de conexiones: (pre, post, distance)
    connections = []
    
    # 1. EXCITATORIAS: 75 locales + 25 distantes cada una
    print("Creando conexiones excitatorias...")
    
    for i in range(N_exc):
        # Distancias desde neurona i a todas las demás excitatorias
        distances_to_exc = distance_matrix[i, :N_exc]
        
        # Excluir la neurona misma
        distances_to_exc[i] = np.inf
        
        # 75 conexiones locales (≤1.5mm)
        local_candidates = np.where(distances_to_exc <= 1.5)[0]
        
        if len(local_candidates) >= 75:
            # Seleccionar 75 al azar de los candidatos locales
            local_targets = np.random.choice(local_candidates, 75, replace=False)
        else:
            # Tomar todos los disponibles si hay menos de 75
            local_targets = local_candidates
            
        for target in local_targets:
            connections.append((i, target, distances_to_exc[target], 'E->E_local'))
        
        # 25 conexiones distantes: axón 12mm a ubicación específica + colaterales 0.5mm
        # 1. Generar ubicación distante aleatoria a ~12mm
        distant_location = generate_distant_location(neuron_positions[i], 12.0)
        
        # 2. Encontrar neuronas excitatorias dentro de 0.5mm de esa ubicación
        distant_candidates = []
        for j in range(N_exc):
            if j != i and j not in local_targets:
                dist_to_location = np.linalg.norm(neuron_positions[j] - distant_location)
                if dist_to_location <= 0.5:
                    distant_candidates.append(j)
        
        # 3. Seleccionar hasta 25 de esos candidatos
        n_distant = min(25, len(distant_candidates))
        if n_distant > 0:
            distant_targets = np.random.choice(distant_candidates, n_distant, replace=False)
            for target in distant_targets:
                # Distancia real neurona->neurona (no a la ubicación intermedia)
                connections.append((i, target, distances_to_exc[target], 'E->E_distant'))
    
    # 2. EXCITATORIAS -> INHIBITORIAS (misma probabilidad)
    print("Creando conexiones E->I...")
    for i in range(N_exc):
        for j in range(N_exc, N_total):  # j son inhibitorias
            if np.random.rand() < 0.09:
                distance = distance_matrix[i, j]
                connections.append((i, j, distance, 'E->I'))
    
    # 3. INHIBITORIAS: 25 conexiones locales cada una
    print("Creando conexiones inhibitorias...")
    
    for i in range(N_exc, N_total):  # i son inhibitorias
        # Conexiones a TODAS las neuronas (E+I) dentro de 0.5mm
        distances_from_i = distance_matrix[i, :]
        local_candidates = np.where(distances_from_i <= 0.5)[0]
        
        # Excluir la neurona misma
        local_candidates = local_candidates[local_candidates != i]
        
        if len(local_candidates) >= 25:
            targets = np.random.choice(local_candidates, 25, replace=False)
        else:
            targets = local_candidates
            
        for target in targets:
            if target < N_exc:
                conn_type = 'I->E'
            else:
                conn_type = 'I->I'
            connections.append((i, target, distances_from_i[target], conn_type))
    
    return connections

# Crear conectividad
connections = create_spatial_connectivity()

# Análisis de conectividad
print(f"\nEstadísticas de conectividad:")
print(f"Total conexiones: {len(connections)}")

# Agrupar por tipo
conn_types = {}
for conn in connections:
    conn_type = conn[3]
    if conn_type not in conn_types:
        conn_types[conn_type] = []
    conn_types[conn_type].append(conn)

for conn_type, conns in conn_types.items():
    distances = [c[2] for c in conns]
    print(f"  {conn_type}: {len(conns)} conexiones, distancia {np.mean(distances):.2f}±{np.std(distances):.2f}mm")

# Verificar reglas del paper
print(f"\nVerificación de reglas:")

# Conexiones por neurona excitatoria
exc_out_connections = np.zeros(N_exc)
for conn in connections:
    if conn[0] < N_exc:  # presináptica excitatoria
        exc_out_connections[conn[0]] += 1

print(f"  Conexiones salientes excitatorias: {np.mean(exc_out_connections):.1f}±{np.std(exc_out_connections):.1f}")
print(f"  Esperado: ~100 (75 locales + 25 distantes + E->I)")

# Conexiones por neurona inhibitoria
inh_out_connections = np.zeros(N_inh)
for conn in connections:
    if conn[0] >= N_exc:  # presináptica inhibitoria
        inh_out_connections[conn[0] - N_exc] += 1

print(f"  Conexiones salientes inhibitorias: {np.mean(inh_out_connections):.1f}±{np.std(inh_out_connections):.1f}")
print(f"  Esperado: ~25")

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Distribución de distancias por tipo
ax = axes[0, 0]
for conn_type, conns in conn_types.items():
    distances = [c[2] for c in conns]
    ax.hist(distances, bins=30, alpha=0.6, label=conn_type, density=True)
ax.set_xlabel('Distancia (mm)')
ax.set_ylabel('Densidad')
ax.set_title('Distancias por Tipo de Conexión')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Conectividad local vs distante (solo E->E)
ax = axes[0, 1]
local_conns = [c for c in connections if c[3] == 'E->E_local']
distant_conns = [c for c in connections if c[3] == 'E->E_distant']

local_dists = [c[2] for c in local_conns]
distant_dists = [c[2] for c in distant_conns]

ax.hist(local_dists, bins=20, alpha=0.7, label=f'Locales (≤1.5mm): {len(local_conns)}')
ax.hist(distant_dists, bins=20, alpha=0.7, label=f'Distantes: {len(distant_conns)}')
ax.axvline(1.5, color='red', linestyle='--', label='Límite local')
ax.set_xlabel('Distancia (mm)')
ax.set_ylabel('Número de conexiones')
ax.set_title('Conexiones E->E: Locales vs Distantes')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Grado de conectividad entrada
ax = axes[0, 2]
in_degree_exc = np.zeros(N_exc)
in_degree_inh = np.zeros(N_inh)

for conn in connections:
    post = conn[1]
    if post < N_exc:
        in_degree_exc[post] += 1
    else:
        in_degree_inh[post - N_exc] += 1

ax.hist(in_degree_exc, bins=20, alpha=0.7, label='Excitatorias')
ax.hist(in_degree_inh, bins=20, alpha=0.7, label='Inhibitorias')
ax.set_xlabel('Conexiones entrantes')
ax.set_ylabel('Número de neuronas')
ax.set_title('Distribución de Grado de Entrada')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Matriz de conectividad (muestra 50x50)
ax = axes[1, 0]
sample_size = 1000
conn_matrix = np.zeros((sample_size, sample_size))

for conn in connections:
    if conn[0] < sample_size and conn[1] < sample_size:
        conn_matrix[conn[0], conn[1]] = 1

im = ax.imshow(conn_matrix, cmap='Blues', aspect='equal')
ax.set_xlabel('Neurona post')
ax.set_ylabel('Neurona pre')
ax.set_title(f'Matriz Conectividad ({sample_size}×{sample_size})')
plt.colorbar(im, ax=ax)

# 5. Conectividad espacial en 2D
ax = axes[1, 1]
# Proyectar neuronas
x_coords = neuron_positions[:, 0]
y_coords = neuron_positions[:, 1]

ax.scatter(x_coords[:N_exc], y_coords[:N_exc], c='blue', s=4, alpha=0.6)
ax.scatter(x_coords[N_exc:], y_coords[N_exc:], c='red', s=8, alpha=0.8)

# Mostrar algunas conexiones (sample)
sample_connections = np.random.choice(len(connections), 200, replace=False)
for idx in sample_connections:
    conn = connections[idx]
    pre, post = conn[0], conn[1]
    ax.plot([x_coords[pre], x_coords[post]], 
           [y_coords[pre], y_coords[post]], 
           'k-', alpha=0.1, linewidth=0.3)

ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_title('Conectividad Espacial (muestra)')
ax.set_aspect('equal')

# 6. Estadísticas por región
ax = axes[1, 2]
# Dividir esfera en regiónes y analizar densidad de conexiones
n_regions = 8
region_connections = np.zeros(n_regions)
angles = np.linspace(0, 2*np.pi, n_regions+1)

for i, conn in enumerate(connections[:1000]):  # Sample
    pre_angle = np.arctan2(y_coords[conn[0]], x_coords[conn[0]])
    if pre_angle < 0:
        pre_angle += 2*np.pi
    region = int(pre_angle / (2*np.pi) * n_regions)
    region_connections[region] += 1

ax.bar(range(n_regions), region_connections)
ax.set_xlabel('Región espacial')
ax.set_ylabel('Conexiones (muestra)')
ax.set_title('Distribución Espacial de Conexiones')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Guardar conectividad para siguiente paso
connection_array = np.array([(c[0], c[1], c[2]) for c in connections])
np.save('spatial_connections.npy', connection_array)

connection_types = np.array([c[3] for c in connections])
np.save('connection_types.npy', connection_types)

print(f"\nDatos guardados:")
print(f"  spatial_connections.npy: (pre, post, distance)")
print(f"  connection_types.npy: tipos de conexión")